In [13]:
# Imports
import pandas as pd
import numpy as np
from numpy.random import randint as some_int
from numpy.random import rand as some_float

In [75]:
def show_table(table):
    print("User_id \tNum_shares \tPrice\n")
    for entry in table:
        print("\t\t".join([str(i) for i in entry]))

## Base Logic for concurrency and trade execution
At any given point in time, the minimum selling price will be > highest bidding price.<br><br><br>
The above state is what remains at any given time in our database.<br>
When a new request comes (bid or ask), then we add that entry to the table if it is not the highest bid/ lowest ask.<br>
If it is the highest bid, then we check to see if it is >= lowest ask, <br>
&nbsp; &nbsp; &nbsp; &nbsp; If it is, we do the transaction (exchange shares and money)<br><br>

If it is the lowest ask, then we check to see if it is <= highest bid, <br>
&nbsp; &nbsp; &nbsp; &nbsp; If it is, we do the transaction (exchange shares and money)<br><br>

Finally, we update our tables to get back into the state where highest bid < lowest ask.<br><br>

The lowest ask is what we show as the price of the stock for anyone looking to buy.<br>
The highest bid is what we show as the price of the stock for anyone looking to sell.

In [128]:
'''
This is the current state space:
Every stock under consideration (xyz) will have 2 tables:
1. Bid requests (offer to buy n shares of xyz stock at p price) -> arranged in descending order
2. Ask requests (offer to sell up to n shares of xyz stock at p price) -> arranged in ascending order
'''

class Bid_Ask:
    def __init__(self):
        
        # Initialize both tables with a blank and then fill in some initial values
        self.bids = []
        self.asks = []

        # The price below is arbitrary, I am adding it to get a table more resembling the real one
        lowest_bid = 50
        bid_spread = int(lowest_bid*0.1)

        for bid in range(some_int(5,10)):
            user_id = 'User' + str(some_int(1,20))
            num_shares = some_int(10,30)
            bid_price = lowest_bid + bid_spread*some_float()

            self.bids.append([user_id, num_shares, bid_price])
        self.re_sort('bids')

        # The price below is arbitrary, I am adding it to get a table more resembling the real one
        lowest_ask = int(max([x[2] for x in self.bids]))+1
        ask_spread = int(lowest_ask*0.1)

        for ask in range(some_int(5,10)):
            user_id = 'User' + str(some_int(11,30))
            num_shares = some_int(10,30)
            ask_price = lowest_ask + ask_spread*some_float()

            self.asks.append([user_id, num_shares, ask_price])

        self.re_sort('asks')
        
        self.entry = []

    def re_sort(self,table_name):
        if table_name == 'bids':
            # Descending order
            self.bids = sorted(self.bids, key = lambda x: -x[2])
        else:
            # Ascending order
            self.asks = sorted(self.asks, key = lambda x: x[2])
            
    def show_table(self, table):
        '''
        table is either a bid table or an ask table.
        The shape is (number_of_entries, 3):
            The 3 columns are User_id, num_shares and price
        '''
        print("User_id \t\tNum_shares \t\tPrice\n\n")
        for entry in table:
            print("\t\t".join([str(i) for i in entry]))

    def show_bids(self):
        print('-'*50)
        print('Bid table sorted in descending order:\n')
        show_table(self.bids)
        print('-'*50)
    
    def show_asks(self):
        print('-'*50)
        print('Ask table sorted in ascending order:\n')
        show_table(self.asks)
        print('-'*50)

    def handle_bid_entry(self):
        
        transaction = {}
        
        # get details of the cheapest seller
        
        lowest_ask = self.asks[0]

        bid_price = self.entry[2]
        lowest_ask_price = lowest_ask[2]

        bid_shares = self.entry[1]
        ask_shares = lowest_ask[1]

        if bid_price >= lowest_ask_price:
            # First checking if the bidding price is greater than cheapest asking price

            transaction['type'] = 'BID'
            transaction['buyer'] = self.entry[0]
            transaction['seller'] = lowest_ask[0]
            transaction['price'] = lowest_ask_price

            # Calculate remaining shares after the transaction
            remaining_shares = ask_shares - bid_shares
            
            if remaining_shares <= 0:
                # If the number of shares to buy > number of shares to sell, then all the available shares will be sold and the bid entry will be updated with a reduced number of bid_shares
                transaction['num_shares'] = ask_shares
                self.entry[1] = -remaining_shares

                # Delete the ask(seller) entry since the seller sold all the shares acc to the lowest ask request
                self.asks = self.asks[1:]
            else:
                # Here, we simply do the transaction and modify the ask entry
                transaction['num_shares'] = bid_shares
                self.entry[1] = 0
                self.asks[0][1] = remaining_shares
        else:
            # This is the block when the bidding price (in the new request) is lower than the cheapest selling price
            # No transaction right now, just add the entry to the bid table
            self.bids.append(list(self.entry))
            self.re_sort('bids')
            self.entry[1] = 0
            
        return transaction
    
    def handle_ask_entry(self):
        
        transaction = {}
        
        highest_bid = self.bids[0]

        ask_price = self.entry[2]
        highest_bid_price = highest_bid[2]

        ask_shares = self.entry[1]
        bid_shares = highest_bid[1]

        if ask_price <= highest_bid_price:
            transaction['type'] = 'ASK'
            transaction['seller'] = self.entry[0]
            transaction['buyer'] = highest_bid[0]
            transaction['price'] = highest_bid_price

            remaining_shares = bid_shares - ask_shares
            if remaining_shares <= 0:
                # If the number of shares to sell > number of shares to buy, then all the available shares will be bought and the ask entry will be updated with a reduced number of ask_shares
                transaction['num_shares'] = bid_shares
                self.entry[1] = -remaining_shares

                # Delete the bid(buyer) entry since the buyer bought all the shares acc to the highest bid request
                self.bids = self.bids[1:]
            else:
                # Here, we simply do the transaction and modify the bid entry
                transaction['num_shares'] = ask_shares
                self.entry[1] = 0
                self.bids[0][1] = remaining_shares
        else:
            # This is the block when the asking price (in the new request) is higher than the highest buying price
            # No transaction right now, just add the entry to the table
            self.asks.append(list(self.entry))
            self.re_sort('asks')
            self.entry[1] = 0
        
        return transaction

        
        
    def add_entry(self, entry, type_of_entry = 'bid'):
        
        self.entry = entry
        
        if type_of_entry == 'bid':
            transaction = self.handle_bid_entry()
        else:
            transaction = self.handle_ask_entry()
                    
        if transaction != {}:
            print("Transaction details:\n",transaction,'\n\n')
        if self.entry[1] != 0:
            # Say you are a buyer who wants to buy 150 shares for Rs. 100 and the lowest selling price is Rs. 90 but only for 50 shares
            # Then we want to execute 1 transaction of selling 50 shares and for Rs.90 and for the remaining 100,
            # We need to run the updated entry for buying 100 shares for Rs.100 now
            self.add_entry(entry, type_of_entry = type_of_entry)
        else:
            pass
                

    def __str__(self):
        print('Current Statespace includes:\n\nCurrent Bid table:\n')
        self.show_bids()
        print('\n\n\nCurrent Ask table:\n')
        self.show_asks()
        return ''



In [129]:
# We are creating an instance corresponding to a particular stock (eg. XYZ shell corporation)

xyz_stock_manager = Bid_Ask()
print(xyz_stock_manager)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User9		27		54.701458730239736
User12		25		54.558852505475116
User18		19		53.46733002633789
User5		20		52.131630142205395
User18		14		51.91300402105757
User7		14		51.06298743084816
User2		26		50.25047877357715
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User25		12		55.197088582291315
User13		25		55.90064868278656
User20		26		56.051707404560716
User27		27		57.500407058802324
User23		23		58.04790729764365
User26		16		59.01961893395346
User16		13		59.61725064060019
User26		28		59.64763862091588
User15		10		59.68787971029525
--------------------------------------------------



In [137]:
'''
1 Bid Request Followed by 1 Ask request, no transactions executed, just adding new entries into table
'''
example_0 = Bid_Ask()
print(example_0)

print('-'*100, '\nNow, we will be adding a bid request\n', '-'*100, '\n')

user = 'Ansh'
num_shares = some_int(1,3)
price = (example_0.asks[0][2] + example_0.asks[1][2])/2.0 - 5
entry = [user, num_shares, price]
print('This is the request that would come from my side when I am submitting a bid request\n', entry,'\n\n\n')
example_0.add_entry(entry, 'bid')
print(example_0)

print('-'*100, '\nNow, we will be adding a ask request\n', '-'*100, '\n')

user = 'Ansh'
num_shares = some_int(1,3)
price = (example_0.bids[0][2] + example_0.bids[1][2])/2.0 + 5
entry = [user, num_shares, price]
print('This is the request that would come from my side when I am submitting a ask request\n', entry,'\n\n\n')
example_0.add_entry(entry, 'ask')
print(example_0)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User3		23		54.6740832476071
User3		24		54.297210602817856
User8		13		53.843700229428926
User14		13		53.44446820349168
User6		21		52.21155130456368
User9		14		52.19832081825495
User3		10		51.72230188350698
User13		12		50.526339980647506
User15		12		50.2700254828038
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User18		19		55.269343621050304
User28		17		55.6961631819022
User14		13		55.881964642846356
User24		17		56.09741349824767
User16		25		59.84200157150116
--------------------------------------------------

---------------------------------------------------------------------------------------------------- 
Now, we will be adding a bid request
 ---------------------------------

In [138]:
'''
Bid request, Number of shares to buy < number of shares to sell (for cheapest seller)
only 1 transaction will happen, no entries added to the table, cheapest seller entry will be modified
'''
example_1 = Bid_Ask()
print(example_1)

print('-'*100, '\nNow, we will be adding a bid/ask request\n', '-'*100, '\n')

user = 'Ansh'
num_shares = example_1.asks[0][1]-some_int(1,5)
price = (example_1.asks[0][2] + example_1.asks[1][2])/2.0
entry = [user, num_shares, price]
print('This is the request that would come from my side when I am submitting a bid request\n', entry,'\n\n\n')
example_1.add_entry(entry, 'bid')
print(example_1)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User4		29		53.901652715082896
User18		17		53.309050058062326
User11		23		53.21317950426664
User6		26		53.20039282575268
User16		17		51.860922843976994
User4		17		51.63956121077759
User9		23		50.84465995340661
User7		18		50.05575801744347
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User22		21		54.55919199091035
User26		20		56.20604880064734
User29		25		56.96292965742163
User17		24		57.00344591879204
User21		15		57.119306184328025
User28		12		57.284734439964964
User17		24		57.76757391737223
--------------------------------------------------

---------------------------------------------------------------------------------------------------- 
Now, we will be adding a bid/ask requ

In [139]:
'''
Bid request, Number of shares to buy = number of shares to sell (for cheapest seller)
only 1 transaction will happen, no entries added to the table, cheapest seller entry will be removed
'''
example_2 = Bid_Ask()
print(example_2)

print('-'*100, '\nNow, we will be adding a bid/ask request\n', '-'*100, '\n')

user = 'Ansh'
num_shares = example_2.asks[0][1]
price = (example_2.asks[0][2] + example_2.asks[1][2])/2.0
entry = [user, num_shares, price]
print('This is the request that would come from my side when I am submitting a bid request\n', entry,'\n\n\n')
example_2.add_entry(entry, 'bid')
print(example_2)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User4		20		53.693426956292896
User4		21		52.95922877274174
User15		28		52.75220336125066
User8		25		52.726330667943245
User14		11		52.52494267804368
User18		11		52.068621732565575
User6		27		51.66059091506282
User12		12		50.987203041788035
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User25		15		54.23699201643095
User21		13		54.30626380289053
User15		20		55.06852992091036
User14		10		56.72287965541974
User22		16		58.46878752633481
User28		22		58.53316611337854
--------------------------------------------------

---------------------------------------------------------------------------------------------------- 
Now, we will be adding a bid/ask request
 -------------------------

In [140]:
'''
Bid request, Number of shares to buy > number of shares to sell (for 3 of the cheapest seller)
only 3 transaction will happen, 3 entries will be deleted from the ask table and 1 entry added to bid table
'''
example_3 = Bid_Ask()
print(example_3)

print('-'*100, '\nNow, we will be adding a bid/ask request\n', '-'*100, '\n')

user = 'Ansh'
num_shares = example_3.asks[0][1]+example_3.asks[1][1]+example_3.asks[2][1]+some_int(1,5)
price = (example_3.asks[2][2] + example_3.asks[3][2])/2.0
entry = [user, num_shares, price]
print('This is the request that would come from my side when I am submitting a bid request\n', entry,'\n\n\n')
example_3.add_entry(entry, 'bid')
print(example_3)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User6		26		54.81361590191962
User6		25		54.17940379379978
User17		15		53.833611748851865
User10		12		53.73693241754174
User10		27		53.298164835537655
User13		15		52.22588491758635
User11		26		51.630251010511124
User19		22		51.30373815676467
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User25		18		56.11144177391834
User21		24		57.97821973036026
User22		14		57.997898025713454
User15		25		58.70702793403619
User23		17		59.248385686337485
User19		28		59.4632781482698
--------------------------------------------------

---------------------------------------------------------------------------------------------------- 
Now, we will be adding a bid/ask request
 -----------------------

In [141]:
'''
Ask request, Number of shares to sell > number of shares to buy (for 3 of the highest bidders)
only 3 transaction will happen, 3 entries will be removed from the bid table and 1 entry will be added to the ask table
'''
example_4 = Bid_Ask()
print(example_4)

print('-'*100, '\nNow, we will be adding a bid/ask request\n', '-'*100, '\n')

user = 'Ansh'
num_shares = example_4.bids[0][1]+example_4.bids[1][1]+example_4.bids[2][1]+some_int(1,5)
price = (example_4.bids[2][2] + example_4.bids[3][2])/2.0
entry = [user, num_shares, price]
print('This is the request that would come from my side when I am submitting an ask request\n', entry,'\n\n\n')
example_4.add_entry(entry, 'ask')
print(example_4)

Current Statespace includes:

Current Bid table:

--------------------------------------------------
Bid table sorted in descending order:

User_id 	Num_shares 	Price

User2		22		54.48591352993978
User7		28		54.14488472447546
User1		25		53.65897099189044
User2		10		53.4906137382745
User16		29		52.17538842327431
User5		11		52.16619924143259
User14		21		51.49208017505843
User19		18		50.68594899215308
--------------------------------------------------



Current Ask table:

--------------------------------------------------
Ask table sorted in ascending order:

User_id 	Num_shares 	Price

User18		24		55.56975641335991
User23		26		57.0552595827035
User20		12		57.3642615721499
User11		12		57.65192275145966
User22		19		58.245503558305515
User15		25		59.272210321426584
--------------------------------------------------

---------------------------------------------------------------------------------------------------- 
Now, we will be adding a bid/ask request
 -------------------------------